In [ ]:
#!pip install pandas
#!pip install numpy
#!pip install nltk
#!pip install stanfordnlp

In [ ]:
data_eng=pd.read_csv('/content/English_final.csv')

In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import stanfordnlp
import spacy
# Make sure you have downloaded the StanfordNLP English model and other essential tools using,
#stanfordnlp.download('en')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
def aspect_sentiment_analysis(txt, stop_words, nlp):
    
    txt = txt.lower() # LowerCasing the given Text
    sentList = nltk.sent_tokenize(txt) # Splitting the text into sentences

    fcluster = []
    totalfeatureList = []
    finalcluster = []
    dic = {}

    for line in sentList:
        newtaggedList = []
        txt_list = nltk.word_tokenize(line) # Splitting up into words
        taggedList = nltk.pos_tag(txt_list) # Doing Part-of-Speech Tagging to each word

        newwordList = []
        flag = 0
        for i in range(0,len(taggedList)-1):
            if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"): # If two consecutive words are Nouns then they are joined together
                newwordList.append(taggedList[i][0]+taggedList[i+1][0])
                flag=1
            else:
                if(flag==1):
                    flag=0
                    continue
                newwordList.append(taggedList[i][0])
                if(i==len(taggedList)-2):
                    newwordList.append(taggedList[i+1][0])

        finaltxt = ' '.join(word for word in newwordList) 
        new_txt_list = nltk.word_tokenize(finaltxt)
        wordsList = [w for w in new_txt_list if not w in stop_words]
        taggedList = nltk.pos_tag(wordsList)

        #print(wordsList)
        #print(taggedList)
        
        doc = nlp(finaltxt) # Object of Stanford NLP Pipeleine
        
        # Getting the dependency relations betwwen the words
        dep_node = []
        for dep_edge in doc.sentences[0].dependencies:
            dep_node.append([dep_edge[2].text, dep_edge[0].index, dep_edge[1]])
        
        #print(dep_node)    
        
        # Coverting it into appropriate format
        for i in range(0, len(dep_node)):
            if (int(dep_node[i][1]) != 0):
              dep_node[i][1] = newwordList[ (int(dep_node[i][1])) - 1 ]
                

        featureList = []
        categories = []
        for i in taggedList:
            if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
                featureList.append(list(i)) # For features for each sentence
                totalfeatureList.append(list(i)) # Stores the features of all the sentences in the text
                categories.append(i[0])

        for i in featureList:
            filist = []
            for j in dep_node:
                if((j[0]==i[0] or j[1]==i[0]) and (j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"])):
                    if(j[0]==i[0]):
                        filist.append(j[1])
                    else:
                        filist.append(j[0])
            fcluster.append([i[0], filist])
            
    for i in totalfeatureList:
        dic[i[0]] = i[1]
    
    for i in fcluster:
        if(dic[i[0]]=="NN"):
            finalcluster.append(i)
        
    return(finalcluster)
    #'''

In [ ]:
#import stanfordnlp
#MODELS_DIR = '.'
#stanfordnlp.download('en', MODELS_DIR) # Download the English models
#nlp = stanfordnlp.Pipeline(models_dir=MODELS_DIR, treebank='en_ewt', use_gpu=True, pos_batch_size=3000)

#nlp = stanfordnlp.Pipeline()

nlp=spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

txt = "The Sound Quality is great but the battery life is very bad."

print(aspect_sentiment_analysis(txt, stop_words, nlp))

# Output: [['soundquality', ['great']], ['batterylife', ['bad']]]


In [ ]:
#import stanfordnlp

#MODELS_DIR = '.'
#stanfordnlp.download('en', MODELS_DIR) # Download the English models
#nlp = stanfordnlp.Pipeline(models_dir=MODELS_DIR, treebank='en_ewt', use_gpu=True, pos_batch_size=3000) # Build the pipeline, specify part-of-speech processor's batch size
#doc = nlp("The SoundQuality is great but the battery life is very bad") # Run the pipeline on input text
#doc.sentences[0].dependencies

In [ ]:
#data_eng

In [ ]:
#MODELS_DIR = '.'
#nlp = stanfordnlp.Pipeline(models_dir=MODELS_DIR, treebank='en_ewt', use_gpu=True, pos_batch_size=3000) 
#nlp=stanfordnlp.Pipeline()

txt = data_eng['Tweet'][0]
nlp=spacy.load('en_core_web_sm')
doc = nlp(txt)

In [ ]:
#for token in doc:
#  print(token, token.dep_, token.head.text)

In [ ]:
def Noun_dependancy_pair(txt):
  txt = txt.lower() # LowerCasing the given Text
  sentList = nltk.sent_tokenize(txt)
  doc = nlp(txt)
  #print(sentList) # Splitting the text into sentences
  fcluster = []
  totalfeatureList = []
  finalcluster = []
  dic = {}
  for line in sentList:
    newtaggedList = []
    txt_list = nltk.word_tokenize(line) #[word for word in line.split()]#nltk.word_tokenize(line) # Splitting up into word
    taggedList = nltk.pos_tag(txt_list) # Doing Part-of-Speech Tagging to each word
  #print(taggedList)
  newwordList = []
  flag = 0
  for i in range(0,len(taggedList)-1):
    if(taggedList[i][1]=="NN" and taggedList[i+1][1]=="NN"): 
      # If two consecutive words are Nouns then they are joined togethe
      newwordList.append(taggedList[i][0]+taggedList[i+1][0])
      flag=1
    else:
      if(flag==1):
        flag=0
        continue
    newwordList.append(taggedList[i][0])
    if(i==len(taggedList)-2):
      newwordList.append(taggedList[i+1][0])
  #print(newwordList)
  finaltxt = ' '.join(word for word in newwordList) 
  #print(finaltxt)
  
  new_txt_list = nltk.word_tokenize(finaltxt) #[word for word in finaltxt.split()] #nltk.word_tokenize(finaltxt)
  #print(new_txt_list)
  wordsList = [w for w in new_txt_list if not w in stop_words]
  #print(wordsList)
  taggedList = nltk.pos_tag(wordsList)
  #print(taggedList)
  dep_node = []
  for token in doc:
    #print(nltk.pos_tag(nltk.word_tokenize(txt)))
    dep_node.append([token.text, token.head.text, token.dep_])
  #print(dep_node)
  featureList = []
  categories = []
  for i in taggedList:
    if(i[1]=='JJ' or i[1]=='NN' or i[1]=='JJR' or i[1]=='NNS' or i[1]=='RB'):
      featureList.append(list(i)) # For features for each sentence
      totalfeatureList.append(list(i)) # Stores the features of all the sentences in the text
      categories.append(i[0])
  
  for i in featureList:
    filist = []
    for j in dep_node:
      if(((j[0] in i[0]) or (j[1] in i[0]))):
        if(j[2] in ["nsubj", "acl:relcl", "obj", "dobj", "agent", "advmod", "amod", "neg", "prep_of", "acomp", "xcomp", "compound"]):
          if(j[0] in i[0]):
            filist.append(j[1])
          else:
            filist.append(j[0])
    fcluster.append([i[0], filist])
  
  for i in totalfeatureList:
    dic[i[0]] = i[1]
  
  ent_lis=[]
  for ent in doc.ents:
    ent_lis.append(ent.text)  
  for i in fcluster:
    if(dic[i[0]]=="NN"):
      finalcluster.append(i)  
  return(finalcluster)       

In [ ]:
#for ent in doc.ents:
#  print(ent.text,ent.label_)

In [ ]:
finalcluster= Noun_dependancy_pair(txt)

In [ ]:
finalcluster

[['samsung', ['follow', 'updates']],
 ['addition', ['series']],
 ['seriesword', ['addition', 'word', 'making']],
 ['series', ['addition', 'word']],
 ['galaxy', ['variant', 'upcoming', 'updates']],
 ['batterycharger', ['updates', 'awesome', 'updates']],
 ['battery', ['awesome', 'updates']],
 ['specstech', ['updates', 'updates']],
 ['specs', ['updates']],
 ['backhandindexpointingrightsmartphone', ['updates', 'mobilephone']],
 ['backhandindexpointingright', ['updates', 'mobilephone']],
 ['mobilephonemobilephoto',
  ['backhandindexpointingright', 'mobilephone', 'follow']],
 ['mobilephone', ['backhandindexpointingright', 'mobilephone', 'follow']],
 ['mobilephoto', ['mobilephone']]]

In [ ]:
#dep_node = []
#for dep_edge in doc.sentences[0].dependencies:
#  dep_node.append([dep_edge[2].text, dep_edge[0].index, dep_edge[1]])
#print(dep_node)

[['The', '3', 'det'], ['Sound', '3', 'compound'], ['Quality', '5', 'nsubj'], ['is', '5', 'cop'], ['great', '0', 'root'], ['but', '12', 'cc'], ['the', '9', 'det'], ['battery', '9', 'compound'], ['life', '12', 'nsubj'], ['is', '12', 'cop'], ['very', '12', 'advmod'], ['bad', '5', 'conj']]


In [ ]:
# Coverting it into appropriate format
#for i in range(0, len(dep_node)):
#  if (int(dep_node[i][1])!= 0):
#    #print(int(dep_node[i][1]))  
#    dep_node[i][1] = newwordList[ (int(dep_node[i][1]))]
                              

In [ ]:
import pandas as pd

afinn_wl_url = ('https://raw.githubusercontent.com'
                '/fnielsen/afinn/master/afinn/data/AFINN-111.txt')

afinn_wl_df = pd.read_csv(afinn_wl_url,
                          header=None, # no column names
                          sep='\t',  # tab sepeated
                          names=['term', 'value']) #new column names

seed = 808 # seed for sample so results are stable

In [ ]:
pos=[]
neg=[]
for i in range(len(afinn_wl_df)):
  if(afinn_wl_df['value'][i]>=0):
    pos.append(afinn_wl_df['term'][i])
  else:
    neg.append(afinn_wl_df['term'][i])  

In [ ]:
# create a list of globally defined positive and negative words to identify sentiment
# sentiment score based on the laxicon neg, pos words
def feature_sentiment(sentence, pos=pos, neg=neg):
    sent_dict = dict()
    sentence = nlp(sentence)
    opinion_words = neg + pos
    debug = 0
    for token in sentence:
        #sentiment=0.01
        #print(token.head)
        # check if the word is an opinion word, then assign sentiment
        if token.text in opinion_words:
            sentiment = 1 if token.text in pos else -1
            # if target is an adverb modifier (i.e. pretty, highly, etc.)
            # but happens to be an opinion word, ignore and pass
            if (token.dep_ == "advmod"):
              continue
            elif (token.dep_ == "amod"):
              sent_dict[token.head.text] = sentiment
          # for opinion words that are adjectives, adverbs, verbs...
            else:
              for child in token.children:
                # if there's a adj modifier (i.e. very, pretty, etc.) add more weight to sentiment
                # This could be better updated for modifiers that either positively or negatively emphasize
                if ((child.dep_ == "amod") or (child.dep_ == "advmod")) and (child.text in opinion_words):
                  sentiment *= 1.5
                  # check for negation words and flip the sign of sentiment
                if child.dep_ == "neg":
                  sentiment *= -1
              for child in token.children:
                # if verb, check if there's a direct object
                if (token.pos_ == "VERB") & (child.dep_ == "dobj"):                        
                  sent_dict[child.text] = sentiment
                  # check for conjugates (a AND b), then add both to dictionary
                  subchildren = []
                  conj = 0
                  for subchild in child.children:
                    if subchild.text == "and":
                      conj=1
                    if (conj == 1) and (subchild.text != "and"):
                      subchildren.append(subchild.text)
                      conj = 0
                  for subchild in subchildren:
                    sent_dict[subchild] = sentiment

                # check for negation
              for child in token.head.children:
                noun = ""
                if ((child.dep_ == "amod") or (child.dep_ == "advmod")) and (child.text in opinion_words):
                  sentiment *= 1.5
                  # check for negation words and flip the sign of sentiment
                if (child.dep_ == "neg"): 
                  sentiment *= -1
                
                # check for nouns
              for child in token.head.children:
                noun = ""
                if (child.pos_ == "NOUN") and (child.text not in sent_dict):
                  noun = child.text
                  # Check for compound nouns
                  for subchild in child.children:
                    if subchild.dep_ == "compound":
                      noun = subchild.text + " " + noun
                    sent_dict[noun] = sentiment
                    debug += 1
    return sent_dict

In [ ]:
sent_dict=feature_sentiment(txt,pos,neg)

In [ ]:
def entity_pair(text):
  #print(text)
  doc=nlp(text)
  lis=[]
  for ent in doc.ents:
    #print(ent.label_, ent.text)
    if(ent.label_=='ORG'):
      lis.append(ent.text.lower())
  
  return(lis)  


In [ ]:
#entity_pair(data_eng['Tweet'][1])

In [ ]:
data_eng['Senti_Dict']=data_eng['Tweet'].apply(lambda x: feature_sentiment(x,pos,neg))
data_eng['Noun_Dependancy_Pair']= data_eng['Tweet'].apply(lambda x: Noun_dependancy_pair(x))
data_eng['Entity_list']=data_eng['Tweet'].apply(lambda x: entity_pair(x))

In [ ]:
finalcluster

[['samsung', ['follow', 'updates']]]

In [ ]:
sent_dict

{'battery': 1, 'updates': 1}

In [ ]:
def entity_sentiment_pairs(text,pos=pos, neg=neg):
  finalcluster=Noun_dependancy_pair(text)
  senti_dict= feature_sentiment(text,pos,neg)
  entity_list= entity_pair(text)
  print(entity_list)
  print(finalcluster)
  print(senti_dict)
  
  filtered_entities=[]
  filtered_pairs=[]
  #all_noun_list=[]
  
  for i in range(len(finalcluster)):
    #all_noun_list.append(finalcluster[i][0])
    if(finalcluster[i][0] in entity_list):
      filtered_entities.append(finalcluster[i][0])
      filtered_pairs.append(finalcluster[i][1])
  
  entity_dict={}
  for i in range(len(filtered_entities)):
    senti=0
    for j in filtered_pairs[i]:
      if(j in senti_dict.keys()):
        senti+=senti_dict[j]
    if(senti >=0):
      entity_dict[filtered_entities[i]]='positive'
    else:
      entity_dict[filtered_entities[i]]='negative'

  return(entity_dict)        



    


In [ ]:
entity_sentiment_pairs('Apple is great,while Samsung sucks')

['apple', 'samsung']
[['apple', ['is']]]
{}


{'apple': 'positive'}

In [ ]:
data_eng['Entity_Sentiment_Pair_Dict']=data_eng['Tweet'].apply(lambda x: entity_sentiment_pairs(x,pos,neg))

In [ ]:
data_eng.to_csv('English_Results.csv')

In [ ]:
data_eng['Entity_Sentiment_Pair_Dict'].value_counts()

In [ ]:
#!pip install bella-tdsa
from bella-tdsa import helper
from bella-tdsa.models.target import TargetDep

target_dep = helper.download_model(TargetDep, 'SemEval 14 Restaurant')
test_example_multi = [{'text' : 'This bread is tasty but the sauce is too rich', 'target': 'sauce', 
                       'spans': [(28, 33)]}]

target_dep.predict(test_example_multi)


SyntaxError: ignored